# **Problem Statement:**

Binary classification using Deep Neural Networks Example: Classify movie reviews into
positive" reviews and "negative" reviews, just based on the text content of the reviews.
Use IMDB dataset

About the dataset:

IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training and 25,000 for testing. So, predict the number of positive and negative reviews using either classification or deep learning algorithms.

Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

pd.set_option('display.max_colwidth', None)

# **Loading the dataset**

In [ ]:
df = pd.read_csv("IMDB Dataset.csv")
df

# **EDA:**

Preprocessing the dataset

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
df['sentiment'].value_counts()

# **Feature Engineering:**

Checking for duplicates

In [ ]:
print(f"Number of Duplicates in the Data: {df.duplicated().sum()}")
print(f"Number of Nulls in the Data: \n {df.isnull().sum()}")

In [ ]:
X = df['review'][:10000]
y = df['sentiment'][:10000]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
y

In [ ]:
le.classes_ # [0, 1]

# **Model Building/ training and testing:**

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
y_train = y_train.astype('float32').reshape((-1, 1))
y_test = y_test.astype('float32').reshape((-1, 1))

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Model Building

In [ ]:
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D, Dense, LSTM, Embedding, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

# max_features = 30000 # Number of words in our vocabulary
max_len = 512 # length of the output vectors

In [ ]:
vectorize_layer = TextVectorization(output_mode = 'int', output_sequence_length = max_len)

In [ ]:
%%time
vectorize_layer.adapt(X_train) 

In [ ]:
vocab_len = len(vectorize_layer.get_vocabulary())
vocab_len

In [ ]:
model = Sequential()
model.add(Input(shape=(1,), dtype = tf.string))
model.add(vectorize_layer)
model.add(Embedding(input_dim = vocab_len, output_dim = 256, input_length= max_len))
model.add(Conv1D(32, 5, strides = 1,activation= 'relu', padding = 'same', kernel_regularizer = l2(0.0001)))
model.add(Conv1D(16, 5, strides = 1,activation= 'relu', padding = 'same', kernel_regularizer = l2(0.0001)))
model.add(MaxPooling1D(4, 1, padding = 'same'))
model.add(tf.keras.layers.Bidirectional(LSTM(64, kernel_regularizer = l2(0.0001))))
model.add(Dense(32, activation = 'relu', kernel_regularizer = l2(0.0001)))
model.add(Dense(16, activation = 'relu', kernel_regularizer = l2(0.0001)))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

In [ ]:
model.compile(optimizer= optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
%%time
hist = model.fit(X_train, y_train,
          epochs= 4, validation_data = (X_test, y_test))

Epoch 1/4
250/250 [==============================] - 279s 1s/step - loss: 0.2107 - accuracy: 0.9251 - val_loss: 0.6006 - val_accuracy: 0.7820
Epoch 2/4
250/250 [==============================] - 267s 1s/step - loss: 0.0738 - accuracy: 0.9804 - val_loss: 0.7193 - val_accuracy: 0.7655
Epoch 3/4
250/250 [==============================] - 290s 1s/step - loss: 0.0363 - accuracy: 0.9924 - val_loss: 1.0525 - val_accuracy: 0.7970
Epoch 4/4
181/250 [====================>.........] - ETA: 1:14 - loss: 0.0204 - accuracy: 0.9960

# **Model Evaluation**

Plotting Model Accuracy Performance

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'])
plt.show()

Plotting Model Loss Performance

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'])
plt.show()

Prediction

In [ ]:
pred = model.predict(['''So, I'm wondering while watching this film, did the producers of this movie get to save money on Sandra Bullock's wardrobe by dragging out her "before" clothes from Miss Congeniality? Did Ms. Bullock also get to sleepwalk through the role by channeling the "before" Gracie Hart? As many reviewers have noted before, the film is very formulaic. Add to that the deja vu viewer experiences with the character of Cassie Maywether as a somewhat darker Gracie Hart with more back story and it rapidly become a snooze fest.<br /><br />The two bad boy serial killers have been done before (and better) in other films. As has the "good guy partner trying to protect his partner despite the evidence" character been seen before. In fact none of the characters in the film ever get beyond two dimensions or try to be anything but trite stereotypes.<br /><br />One last peeve - using the term serial killer is false advertising. Murdering one person - even if it's a premeditated murder - does not make you a serial killer. You may have the potential to become a serial killer but you are not a serial killer or even a spree killer.''',])

In [ ]:
pred > 0.5

In [ ]:
pred2 = model.predict(['''Petter Mattei's "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case with most of the people we encounter.<br /><br />The acting is good under Mr. Mattei's direction. Steve Buscemi, Rosario Dawson, Carol Kane, Michael Imperioli, Adrian Grenier, and the rest of the talented cast, make these characters come alive.<br /><br />We wish Mr. Mattei good luck and await anxiously for his next work.'''])

In [ ]:
pred2 > 0.5

In [ ]:
pred2

Defining a function for inference, in this function we will assume a negative sentiment if prediction not greater than 0.5 and positive sentiment if prediction is greater than 0.5

In [ ]:
def predict_sentiment(text, model):
    text = [text]
    pred = model.predict(text)
    print(pred)
    if pred > 0.5:
        return "Positive"
    else:
        return "Negative"

In [ ]:
pst_txt = '''One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.'''
predict_sentiment(pst_txt, model)

In [ ]:
ngt_txt ='''Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.'''
predict_sentiment(ngt_txt, model)

In [ ]:
ngt_txt2 = '''It's become extremely difficult to find a good horror movie anymore, thought this movie was a good thriller.<br /><br />Could have had better production values but what kept me going was the suspense and the twists. I had real reservations before seeing this movie (because of the cover). I was afraid that it would be excessively bloody and gory. I was wrong.<br /><br />Although there is a lot of scary parts, there is a lot of suspense and drama too.<br /><br />The acting in Dead Line was better than what you would expect from a micro budget horror flick. The characters were believable<br /><br />The movie is really thrilling and quite scary at moments so it makes you grab your seat until the ending credits roll<br /><br />Because of its production values (the sound is not very good for example) 8/10.'''
predict_sentiment(ngt_txt2, model)

In [ ]:
text3 = '''This movie is incredible. If you have the chance, watch it. Although, a warning, you'll cry your eyes out. I do, every time I see it, and I own it and have watched it many times. The performances are outstanding. It deals with darkness and pain and loss, but there is hope. This movie made me look at the world differently: vicarious experience, according to my English teacher. Also, if you've seen it, note the interesting use of shadows and light. Home room is a phenomenal movie, and I rate it 10/10 - for real - because of the excellent acting, amazing plot, and heart-wrenching dialogue. Very tense, very moving. Doesn't give all the answers, but makes many good points about humankind'''
predict_sentiment(text3, model)

Loading the model


In [ ]:
txt = '''What a frustrating movie. A small Southern town is overflowing with possibilities for exploring the complexities of interpersonal relationships and dark underbellies hidden beneath placid surfaces, as anyone who has read anything by Carson McCullers already knows. This does none of that. Instead, the writers settled for cutesy twinkles, cheap warm fuzzies and banal melodrama. The thing looks like a made-for-TV movie, and was directed with no particular distinction, but it's hard to imagine what anyone could have done to make this material interesting.<br /><br />The most frustrating aspect, though, is the fact that there are a lot of extremely competent and appealing actors in this cast, all trying gamely to make the best of things and do what they can with this--well, there's no other word for it--drivel. A tragic waste of talent, in particular that of the great Stockard Channing.'''
predict_sentiment(txt, model)